In [0]:
import torch
import pandas as pd
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
import io
from torch import nn, optim
import torch.nn.functional as F

In [2]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving intelia.xls to intelia (2).xls
User uploaded file "intelia.xls" with length 1635840 bytes


In [3]:
data= pd.read_excel(io.BytesIO(uploaded['intelia.xls']))
data.drop(['Unnamed: 0', 'MouseID'], axis=1, inplace=True)
sample = np.random.choice(data.index, size=int(len(data)*0.7), replace=False)
train_data, test_data = data.iloc[sample], data.drop(sample)
print("Number of training samples is", len(train_data))
print("Number of testing samples is", len(test_data))
print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_data.shape), 
      "\nTest set: \t\t{}".format(test_data.shape))

Number of training samples is 756
Number of testing samples is 324
			Feature Shapes:
Train set: 		(756, 84) 
Test set: 		(324, 84)


In [0]:
features = np.array(train_data.drop('class', axis=1))
targets = np.array(train_data['class'])
features_test = np.array(test_data.drop('class', axis=1))
targets_test = np.array(test_data['class'])
#print(features[:10])
#print(targets[:10])

In [0]:
train_data = TensorDataset(torch.from_numpy(features), torch.from_numpy(targets))
test_data = TensorDataset(torch.from_numpy(features_test), torch.from_numpy(targets_test))
batch_size = 100

train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [6]:
#Checking if GPU is available
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

Training on GPU.


In [0]:
#Building the Simple Neural network
class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(83, 100)
        self.fc2 = nn.Linear(100, 50)
        self.fc3 = nn.Linear(50, 8)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.log_softmax(self.fc3(x), dim=1)
        
        return x

In [14]:
#Training and Evaluating the Network
model = Classifier()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.03)
epochs = 5
train_losses, test_losses = [], []
for e in range(epochs):
    running_loss = 0
    for features, targets in train_loader:
        log_ps = model(features.float())
        loss = criterion(log_ps, targets)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
            
    else:
        test_loss = 0
        accuracy = 0
        
        
        with torch.no_grad():
            model.eval()
            for features_test, targets_test in test_loader:
                log_ps = model(features_test.float())
                test_loss += criterion(log_ps, targets_test)
                
                ps = torch.exp(log_ps)
                top_p, top_class = ps.topk(1, dim=1)
                equals = top_class == targets_test.view(*top_class.shape)
                accuracy += torch.mean(equals.type(torch.FloatTensor))
        
        model.train()
        
        train_losses.append(running_loss/len(train_loader))
        test_losses.append(test_loss/len(test_loader))

        print("Epoch: {}/{}.. ".format(e+1, epochs),
              "Training Loss: {:.3f}.. ".format(train_losses[-1]),
              "Test Loss: {:.3f}.. ".format(test_losses[-1]),
              "Test Accuracy: {:.3f}".format(accuracy/len(test_loader)))

Epoch: 1/5..  Training Loss: 2.140..  Test Loss: 1.886..  Test Accuracy: 0.389
Epoch: 2/5..  Training Loss: 1.540..  Test Loss: 1.037..  Test Accuracy: 0.629
Epoch: 3/5..  Training Loss: 0.673..  Test Loss: 0.353..  Test Accuracy: 0.940
Epoch: 4/5..  Training Loss: 0.126..  Test Loss: 0.018..  Test Accuracy: 1.000
Epoch: 5/5..  Training Loss: 0.008..  Test Loss: 0.005..  Test Accuracy: 1.000
